In [1]:
from importlib import reload
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style='whitegrid')
import ard_logging
import logging
ard_logging.init(force_debug=False)
LOG = logging.getLogger(__name__)
LOG.info('Hello from notebook')

import ard_fit_pipelines as fp
reload(fp.core); reload(fp.transforms); reload(fp)

logging.getLogger('ard_fit_pipelines').setLevel(logging.DEBUG)

2022-06-18 22:47:12,763:    INFO:      ard_logging: MainProcess 1788: Logging initialized.
2022-06-18 22:47:12,765:    INFO:         __main__: MainProcess 1788: Hello from notebook


In [3]:
df = sns.load_dataset('diamonds')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [4]:
reload(fp.transforms)
t = fp.transforms.CopyColumns(cols=['price', 'x'], dest_cols=['price_train', 'x_copy'])
display(t)
display(t.apply(df.head()))

CopyColumns(cols=('price', 'x'), dest_cols=('price_train', 'x_copy'))

2022-06-18 20:07:51,808:   DEBUG: ard_fit_pipelines.core: MainProcess 143: Applying Transform to 5 rows of data: CopyColumns(cols=('price', 'x'), dest_cols=('price_train', 'x_copy'))


,carat,cut,color,clarity,depth,table,price,x,y,z,price_train,x_copy
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,326,3.95
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,326,3.89
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31,327,4.05
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63,334,4.20
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75,335,4.34


In [9]:
t = fp.transforms.CopyColumns(cols=fp.core.hp('train_response_col'), dest_cols=['price_train'])
t.bind_params({
    'train_response_col': 'price'
})
t

CopyColumns(cols=(hp(name='train_response_col'),), dest_cols=('price_train',))

In [10]:
t.bound_params

{'cols': (hp(name='train_response_col'),), 'dest_cols': ('price_train',)}

In [30]:
t.bound_params = 1
#object.__setattr__(t, 'bound_params', 1)

In [86]:
import functools

class Decorator:
    "Decorator docstr"
    def __init__(self, cls):
        functools.update_wrapper(self, cls)
        self.cls = cls
        
    def __call__(self, *args, **kwargs):
        # forward to wrapped class's __init__
        print('Deco')
        return self.cls(*args, **kwargs)
    
@Decorator
class MyClass:
    "My class docstr"
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return f'MyClass({repr(self.x)}, {repr(self.y)})'
    
display(MyClass)
help(MyClass)
display(MyClass(1, 2))

def decorator2(cls, foo=None):
    orig_init = cls.__init__
    cls_name = cls.__name__
    @functools.wraps(orig_init)
    def new_init(self, *args, **kwargs):
        print('Deco2')
        self.foo = foo
        return orig_init(self, *args, **kwargs)
    cls.__init__ = new_init
    
    class InnerClass:
        "Placeholder"
        myvar = 2
        
    inner_cls_name = f'Fit{cls_name}'
    InnerClass.__name__ = inner_cls_name
    InnerClass.__doc__ = f"The fit version of {cls_name}"
        
    setattr(cls, inner_cls_name, InnerClass)
        
    return cls
    
@decorator2k
class MyClass2:
    "My class docstr"
    def __init__(self, x, y):
        """
        Init docstr.
        """
        self.x = x
        self.y = y
    def __repr__(self):
        return f'MyClass2({repr(self.x)}, {repr(self.y)})'
    
display(MyClass2)
help(MyClass2)
display(MyClass2(1, 2))
display(dir(MyClass2(1,2)))

Help on Decorator in module __main__:

MyClass = <__main__.Decorator object>
    My class docstr

Deco


MyClass(1, 2)

TypeError: decorator2() missing 1 required positional argument: 'cls'

In [75]:
help(MyClass2.FitMyClass2)

Help on class FitMyClass2 in module __main__:

class FitMyClass2(builtins.object)
 |  The fit version of MyClass2
 |  
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  myvar = 2



In [111]:
functools.wraps?

Signature:
functools.wraps(
    wrapped,
    assigned=('__module__', '__name__', '__qualname__', '__doc__', '__annotations__'),
    updated=('__dict__',),
)
Docstring:
Decorator factory to apply update_wrapper() to a wrapper function

Returns a decorator that invokes update_wrapper() with the decorated
function as the wrapper argument and the arguments to wraps() as the
remaining arguments. Default arguments are as for update_wrapper().
This is a convenience function to simplify applying partial() to
update_wrapper().
File:      /usr/lib/python3.9/functools.py
Type:      function


In [26]:
import copy
copy.copy?

Signature: copy.copy(x)
Docstring:
Shallow copy operation on arbitrary Python objects.

See the module's __doc__ string for more info.
File:      /usr/lib/python3.9/copy.py
Type:      function


In [33]:
reload(fp.core); reload(fp.transforms); reload(fp)

@fp.core.transform
class Foo:
    """
    Hot stuff.
    """
    x: int = 1
    
    def __init__(self, x=None):
        """
        Constructor help.
        """
        if x is not None:
            self.x = x
            
    def fit(self, df: pd.DataFrame):
        "Fit docstr"
        return len(df)
    
    def __repr__(self):
        return f'Foo({self.x=})'
    
help(Foo)
Foo?

Help on class Foo in module __main__:

class Foo(builtins.object)
 |  Hot stuff.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, x=None)
 |      Constructor help.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  fit(self, df: pandas.core.frame.DataFrame)
 |      Fit docstr
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __annotations__ = {'x': <class 'int'>}
 |  
 |  __wrapped__ = <class '__main__.Foo'>
 |      Hot stuff.
 |  
 |  
 |  x = 1



Init signature: Foo(self, x=None)
Docstring:      Hot stuff.
Init docstring: Constructor help.
Type:           type
Subclasses:     


In [34]:
d = Foo(2)
display(d)
d.fit(df)

Foo(self.x=2)

bleh


53940

In [23]:
import attrs
@attrs.define
class Bar:
    "Bar docstr"
    x: int = 1
    def fit(self, df: pd.DataFrame):
        "Fit docstr"
        return len(df)
    
help(Bar)
Bar?

Help on class Bar in module __main__:

class Bar(builtins.object)
 |  Bar(x: int = 1) -> None
 |  
 |  Bar docstr
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Method generated by attrs for class Bar.
 |  
 |  __getstate__ = slots_getstate(self)
 |      Automatically created by attrs.
 |  
 |  __init__(self, x: int = 1) -> None
 |      Method generated by attrs for class Bar.
 |  
 |  __ne__(self, other)
 |      Method generated by attrs for class Bar.
 |  
 |  __repr__(self)
 |      Method generated by attrs for class Bar.
 |  
 |  __setstate__ = slots_setstate(self, state)
 |      Automatically created by attrs.
 |  
 |  fit(self, df: pandas.core.frame.DataFrame)
 |      Fit docstr
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  x
 |  
 |  ----------------------------------------------------------------------

Init signature: Bar(x: int = 1) -> None
Docstring:      Bar docstr
Init docstring: Method generated by attrs for class Bar.
Type:           type
Subclasses:     
